## Pembenaman

Dalam contoh sebelumnya, kita menggunakan vektor bag-of-words berdimensi tinggi dengan panjang `vocab_size`, dan kita secara eksplisit menukar vektor representasi kedudukan berdimensi rendah kepada representasi satu-hot yang jarang. Representasi satu-hot ini tidak cekap dari segi memori. Selain itu, setiap perkataan dianggap secara bebas antara satu sama lain, jadi vektor yang dikodkan satu-hot tidak menggambarkan persamaan semantik antara perkataan.

Dalam unit ini, kita akan terus meneroka dataset **News AG**. Untuk memulakan, mari kita muatkan data dan dapatkan beberapa definisi daripada unit sebelumnya.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Apa itu embedding?

Idea **embedding** adalah untuk mewakili perkataan menggunakan vektor padat berdimensi rendah yang mencerminkan makna semantik perkataan tersebut. Kita akan bincangkan kemudian bagaimana untuk membina embedding perkataan yang bermakna, tetapi buat masa ini anggaplah embedding sebagai cara untuk mengurangkan dimensi vektor perkataan.

Jadi, lapisan embedding mengambil perkataan sebagai input, dan menghasilkan vektor output dengan `embedding_size` yang ditentukan. Dalam satu aspek, ia sangat mirip dengan lapisan `Dense`, tetapi bukannya mengambil vektor one-hot encoded sebagai input, ia mampu mengambil nombor perkataan.

Dengan menggunakan lapisan embedding sebagai lapisan pertama dalam rangkaian kita, kita boleh beralih daripada model bag-of-words kepada model **embedding bag**, di mana kita mula-mula menukar setiap perkataan dalam teks kita kepada embedding yang sepadan, dan kemudian mengira beberapa fungsi agregat ke atas semua embedding tersebut, seperti `sum`, `average` atau `max`.

![Imej menunjukkan pengelasan embedding untuk lima perkataan dalam urutan.](../../../../../translated_images/ms/embedding-classifier-example.b77f021a7ee67eee.webp)

Rangkaian neural pengelasan kita terdiri daripada lapisan-lapisan berikut:

* Lapisan `TextVectorization`, yang mengambil string sebagai input, dan menghasilkan tensor nombor token. Kita akan menentukan saiz perbendaharaan kata `vocab_size` yang munasabah, dan mengabaikan perkataan yang jarang digunakan. Bentuk input akan menjadi 1, dan bentuk output akan menjadi $n$, kerana kita akan mendapatkan $n$ token sebagai hasilnya, setiap satu mengandungi nombor dari 0 hingga `vocab_size`.
* Lapisan `Embedding`, yang mengambil $n$ nombor, dan mengurangkan setiap nombor kepada vektor padat dengan panjang tertentu (100 dalam contoh kita). Oleh itu, tensor input dengan bentuk $n$ akan ditukar kepada tensor $n\times 100$.
* Lapisan agregasi, yang mengambil purata tensor ini sepanjang paksi pertama, iaitu ia akan mengira purata semua $n$ tensor input yang sepadan dengan perkataan yang berbeza. Untuk melaksanakan lapisan ini, kita akan menggunakan lapisan `Lambda`, dan memasukkan fungsi untuk mengira purata. Output akan mempunyai bentuk 100, dan ia akan menjadi representasi numerik bagi keseluruhan urutan input.
* Pengelasan linear `Dense` terakhir.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Dalam cetakan `summary`, dalam lajur **output shape**, dimensi tensor pertama `None` merujuk kepada saiz minibatch, dan dimensi kedua merujuk kepada panjang urutan token. Semua urutan token dalam minibatch mempunyai panjang yang berbeza. Kita akan bincangkan cara mengatasinya dalam bahagian seterusnya.

Sekarang mari kita latih rangkaian:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Nota** bahawa kami sedang membina penvektoran berdasarkan subset data. Ini dilakukan untuk mempercepatkan proses, dan ia mungkin menyebabkan situasi di mana tidak semua token daripada teks kami terdapat dalam perbendaharaan kata. Dalam kes ini, token tersebut akan diabaikan, yang mungkin mengakibatkan ketepatan yang sedikit lebih rendah. Walau bagaimanapun, dalam kehidupan sebenar, subset teks sering memberikan anggaran perbendaharaan kata yang baik.


### Menguruskan saiz urutan pembolehubah

Mari kita fahami bagaimana latihan berlaku dalam minibatch. Dalam contoh di atas, tensor input mempunyai dimensi 1, dan kita menggunakan minibatch sepanjang 128, jadi saiz sebenar tensor adalah $128 \times 1$. Walau bagaimanapun, bilangan token dalam setiap ayat adalah berbeza. Jika kita menggunakan lapisan `TextVectorization` pada satu input, bilangan token yang dikembalikan adalah berbeza, bergantung pada bagaimana teks itu ditokenkan:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Namun, apabila kita menggunakan penvektor pada beberapa jujukan, ia perlu menghasilkan tensor berbentuk segi empat, jadi ia mengisi elemen yang tidak digunakan dengan token PAD (yang dalam kes kita adalah sifar):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Di sini kita dapat melihat penjelmaan:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Nota**: Untuk meminimumkan jumlah padding, dalam beberapa kes adalah masuk akal untuk menyusun semua jujukan dalam dataset mengikut urutan panjang yang meningkat (atau, lebih tepat lagi, bilangan token). Ini akan memastikan setiap minibatch mengandungi jujukan dengan panjang yang serupa.


## Perwakilan Semantik: Word2Vec

Dalam contoh sebelumnya, lapisan embedding belajar memetakan perkataan kepada perwakilan vektor, namun perwakilan ini tidak mempunyai makna semantik. Akan lebih baik jika kita dapat belajar perwakilan vektor di mana perkataan yang serupa atau sinonim mempunyai vektor yang dekat antara satu sama lain berdasarkan jarak vektor tertentu (contohnya jarak Euclidean).

Untuk mencapai itu, kita perlu melatih model embedding kita terlebih dahulu pada koleksi teks yang besar menggunakan teknik seperti [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ia berdasarkan dua seni bina utama yang digunakan untuk menghasilkan perwakilan teragih bagi perkataan:

 - **Continuous bag-of-words** (CBoW), di mana kita melatih model untuk meramal satu perkataan berdasarkan konteks sekeliling. Diberikan ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, matlamat model adalah untuk meramal $W_0$ daripada $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** adalah bertentangan dengan CBoW. Model menggunakan tetingkap perkataan konteks sekeliling untuk meramal perkataan semasa.

CBoW lebih pantas, manakala skip-gram lebih perlahan tetapi lebih baik dalam mewakili perkataan yang jarang digunakan.

![Imej menunjukkan algoritma CBoW dan Skip-Gram untuk menukar perkataan kepada vektor.](../../../../../translated_images/ms/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Untuk mencuba embedding Word2Vec yang telah dilatih terlebih dahulu pada dataset Google News, kita boleh menggunakan pustaka **gensim**. Di bawah ini, kita mencari perkataan yang paling serupa dengan 'neural'.

> **Nota:** Apabila anda mula-mula mencipta vektor perkataan, memuat turun mereka mungkin mengambil masa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kita juga boleh mengekstrak penjelmaan vektor daripada perkataan, untuk digunakan dalam melatih model klasifikasi. Penjelmaan tersebut mempunyai 300 komponen, tetapi di sini kami hanya menunjukkan 20 komponen pertama vektor untuk kejelasan:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Perkara hebat tentang pengekodan semantik ialah anda boleh memanipulasi pengekodan vektor berdasarkan semantik. Sebagai contoh, kita boleh meminta untuk mencari perkataan yang pengekodan vektornya sedekat mungkin dengan perkataan *raja* dan *wanita*, dan sejauh mungkin dari perkataan *lelaki*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Contoh di atas menggunakan beberapa keajaiban dalaman GenSym, tetapi logik asasnya sebenarnya agak mudah. Satu perkara menarik tentang embedding ialah anda boleh melakukan operasi vektor biasa pada vektor embedding, dan itu akan mencerminkan operasi pada **makna** perkataan. Contoh di atas boleh dinyatakan dalam bentuk operasi vektor: kita mengira vektor yang sepadan dengan **KING-MAN+WOMAN** (operasi `+` dan `-` dilakukan pada representasi vektor perkataan yang sepadan), dan kemudian mencari perkataan paling hampir dalam kamus kepada vektor tersebut:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTA**: Kami terpaksa menambah pekali kecil pada vektor *man* dan *woman* - cuba keluarkan pekali tersebut untuk melihat apa yang berlaku.

Untuk mencari vektor yang paling hampir, kami menggunakan mekanisme TensorFlow untuk mengira vektor jarak antara vektor kami dan semua vektor dalam perbendaharaan kata, dan kemudian mencari indeks perkataan minimum menggunakan `argmin`.


Walaupun Word2Vec kelihatan seperti cara yang hebat untuk menyatakan semantik perkataan, ia mempunyai banyak kelemahan, termasuk yang berikut:

* Kedua-dua model CBoW dan skip-gram adalah **predictive embeddings**, dan mereka hanya mengambil kira konteks tempatan. Word2Vec tidak memanfaatkan konteks global.
* Word2Vec tidak mengambil kira **morfologi** perkataan, iaitu hakikat bahawa makna perkataan boleh bergantung pada bahagian-bahagian tertentu dalam perkataan, seperti akar perkataan.

**FastText** cuba mengatasi kelemahan kedua ini, dan membina atas Word2Vec dengan mempelajari representasi vektor untuk setiap perkataan dan n-gram aksara yang terdapat dalam setiap perkataan. Nilai-nilai representasi ini kemudian dirata-rata menjadi satu vektor pada setiap langkah latihan. Walaupun ini menambah banyak pengiraan tambahan semasa prapemodelan, ia membolehkan embeddings perkataan menyandikan maklumat sub-perkataan.

Kaedah lain, **GloVe**, menggunakan pendekatan yang berbeza untuk embeddings perkataan, berdasarkan faktorisasi matriks konteks-perkataan. Pertama, ia membina matriks besar yang mengira bilangan kejadian perkataan dalam pelbagai konteks, dan kemudian ia cuba mewakili matriks ini dalam dimensi yang lebih rendah dengan cara yang meminimumkan kehilangan rekonstruksi.

Perpustakaan gensim menyokong embeddings perkataan ini, dan anda boleh mencuba dengan menukar kod pemuatan model di atas.


## Menggunakan embedding pralatih dalam Keras

Kita boleh mengubah contoh di atas untuk mengisi matriks dalam lapisan embedding kita dengan embedding semantik, seperti Word2Vec. Perbendaharaan kata embedding pralatih dan korpus teks kemungkinan besar tidak sepadan, jadi kita perlu memilih salah satu. Di sini kita meneroka dua pilihan yang mungkin: menggunakan perbendaharaan kata tokenizer, dan menggunakan perbendaharaan kata daripada embedding Word2Vec.

### Menggunakan perbendaharaan kata tokenizer

Apabila menggunakan perbendaharaan kata tokenizer, beberapa perkataan daripada perbendaharaan kata akan mempunyai embedding Word2Vec yang sepadan, dan beberapa akan tiada. Memandangkan saiz perbendaharaan kata kita adalah `vocab_size`, dan panjang vektor embedding Word2Vec adalah `embed_size`, lapisan embedding akan diwakili oleh matriks berat dengan bentuk `vocab_size`$\times$`embed_size`. Kita akan mengisi matriks ini dengan melalui perbendaharaan kata:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Untuk perkataan yang tidak terdapat dalam perbendaharaan kata Word2Vec, kita boleh sama ada meninggalkannya sebagai sifar, atau menghasilkan vektor rawak.

Sekarang kita boleh mendefinisikan lapisan embedding dengan berat yang telah dilatih:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Nota**: Perhatikan bahawa kami menetapkan `trainable=False` semasa mencipta `Embedding`, yang bermaksud kami tidak melatih semula lapisan Embedding. Ini mungkin menyebabkan ketepatan sedikit lebih rendah, tetapi ia mempercepatkan proses latihan.

### Menggunakan perbendaharaan kata embedding

Satu masalah dengan pendekatan sebelumnya ialah perbendaharaan kata yang digunakan dalam TextVectorization dan Embedding adalah berbeza. Untuk mengatasi masalah ini, kita boleh menggunakan salah satu daripada penyelesaian berikut:
* Melatih semula model Word2Vec pada perbendaharaan kata kita.
* Memuatkan dataset kita dengan perbendaharaan kata daripada model Word2Vec yang telah dilatih. Perbendaharaan kata yang digunakan untuk memuatkan dataset boleh ditentukan semasa proses pemuatan.

Pendekatan kedua kelihatan lebih mudah, jadi mari kita laksanakan. Pertama sekali, kita akan mencipta lapisan `TextVectorization` dengan perbendaharaan kata yang ditentukan, diambil daripada embedding Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Perpustakaan pemetaan perkataan gensim mengandungi fungsi yang mudah, `get_keras_embeddings`, yang akan secara automatik mencipta lapisan pemetaan Keras yang sepadan untuk anda.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Salah satu sebab kita tidak melihat ketepatan yang lebih tinggi adalah kerana beberapa perkataan daripada set data kita tiada dalam kosa kata GloVe yang telah dilatih, dan oleh itu ia pada dasarnya diabaikan. Untuk mengatasi ini, kita boleh melatih penjelmaan kita sendiri berdasarkan set data kita.


## Pembenaman Kontekstual

Satu kelemahan utama dalam representasi pembenaman pralatih tradisional seperti Word2Vec adalah hakikat bahawa, walaupun ia dapat menangkap sebahagian makna sesuatu perkataan, ia tidak dapat membezakan antara makna yang berbeza. Ini boleh menyebabkan masalah dalam model hiliran.

Sebagai contoh, perkataan 'play' mempunyai makna yang berbeza dalam dua ayat berikut:
- Saya pergi ke sebuah **play** di teater.
- John mahu **play** dengan kawan-kawannya.

Pembenaman pralatih yang kita bincangkan sebelum ini mewakili kedua-dua makna perkataan 'play' dalam pembenaman yang sama. Untuk mengatasi kelemahan ini, kita perlu membina pembenaman berdasarkan **model bahasa**, yang dilatih menggunakan korpus teks yang besar, dan *tahu* bagaimana perkataan boleh disusun dalam konteks yang berbeza. Perbincangan tentang pembenaman kontekstual adalah di luar skop tutorial ini, tetapi kita akan kembali kepada topik ini apabila membincangkan model bahasa dalam unit seterusnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
